In [107]:
import pandas as pd
import os

In [149]:


def clean_df(df):
    try:
        df.columns = ['Party', 'Member', 'Accomodation - Wellington',
        'Accomodation - Non-Wellington', 'Air Travel', 'Surface Travel',
        'VIP Transport', 'Grand Total', 'Inter-Parliamentary Office of the Clerk', 'Quarter Ending']
    except:
        df.columns = ['Party', 'Member', 'Accomodation - Wellington',
        'Accomodation - Non-Wellington', 'Air Travel', 'Surface Travel',
        'Grand Total', 'Inter-Parliamentary Office of the Clerk', 'Quarter Ending']
        df.insert(6, "VIP Transport", "")


    df['Party'] = df['Party'].fillna(method='ffill')
    df = df[df["Party"].str.contains("Total|[0-9]") == False]
    df['Member'] = df['Member'].str.replace(' \([0-9]\)', '')
    return df







In [156]:
def append_quarter(filename, df):
    if filename.split()[-1][0].isnumeric():
        file_quarter = str(filename.split()[-2]+" "+filename.split()[-1]).replace('.xlsx', '')
    else:
        file_quarter = str(filename.split()[-1]+" "+filename.split()[0]).replace('.xlsx', '')
    df['Quarter Ending'] = file_quarter
    return df

In [93]:
df = append_quarter(filename, df)

In [157]:
data = []
final_df = pd.DataFrame(data, columns=['Party', 'Member', 'Accomodation - Wellington',
    'Accomodation - Non-Wellington', 'Air Travel', 'Surface Travel',
    'VIP Transport', 'Grand Total', 'Inter-Parliamentary Office of the Clerk', 'Quarter Ending'])

print(final_df)

Empty DataFrame
Columns: [Party, Member, Accomodation - Wellington, Accomodation - Non-Wellington, Air Travel, Surface Travel, VIP Transport, Grand Total, Inter-Parliamentary Office of the Clerk, Quarter Ending]
Index: []


In [158]:
for filename2 in os.listdir("Downloads/"):
    print(filename2)
    if filename2.endswith(".xlsx") and filename2.startswith("2"): 
         df = pd.read_excel ('Downloads/'+filename2, skiprows=2, header=[0,1])
         df = append_quarter(filename2, df)
         df = clean_df(df)
         final_df = final_df.append(df)
    else:
        pass

2020 Members' expense disclosure from 1 April to 30 June 2020.xlsx
2015 Members' expense disclosure from 1 July to 30 September 2015.xlsx
2019 Members' expenses disclosure 1 April to 30 June.xlsx


/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Member'] = df['Member'].str.replace(' \([0-9]\)', '')
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Member'] = df['Member'].str.replace(' \([0-9]\)', '')
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/3793909653.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: FutureWarning: 

2017 Members' expense disclosure from 1 April to 30 June 2017.xlsx
.DS_Store
2019 Members' expenses disclosure 1 January to 31 March.xlsx
2016 Members' expense disclosure from 1 July to 30 September 2016.xlsx
~$2021 Members' expense disclosure from 1 January to 31 March 2021.xlsx
2021 Members' expense disclosure from 1 January to 31 March 2021.xlsx


/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Member'] = df['Member'].str.replace(' \([0-9]\)', '')
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Member'] = df['Member'].str.replace(' \([0-9]\)', '')
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/3793909653.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: FutureWarning: 

2020 Members' expense disclosures from 1 October to 31 December.xlsx
2018 Members' expense disclosure from 1 July to 30 September 2018.xlsx
2019 Members' expense disclosure 1 July to 30 September.xlsx
2020 Members' expense disclosure from 1 July to 30 September.xlsx


...

2018 Members' expense disclosure from 1 October to 31 December 2018.xlsx
2021 Members' expense disclosure from 1 April to 30 June 2021.xlsx
2021 Members' expense disclosure from 1 Oct to 30 Dec 2021.xlsx
2015 Members' expense disclosure from 1 October to 31 December 2015.xlsx


/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Member'] = df['Member'].str.replace(' \([0-9]\)', '')
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Member'] = df['Member'].str.replace(' \([0-9]\)', '')
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/3793909653.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: FutureWarning: 

2016 Members' expense disclosure from 1 April to 30 June 2016.xlsx
2019 Members' expense disclosure from 1 October to 31 December.xlsx
2018 Members' expense disclosure from 1 January to 31 March 2018.xlsx
2020 Members' expense disclosure from 1 January to 31 March.xlsx


/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Member'] = df['Member'].str.replace(' \([0-9]\)', '')
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/3793909653.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Member'] = df['Member'].str.replace(' \([0-9]\)', '')
/var/folders/8v/y2_25bgj5j52ndzgmk478vb80000gn/T/ipykernel_734/54068792.py:15: SettingWithCopy

2016 Members' expense disclosure from 1 January to 31 March 2016.xlsx
2021 Members' expense disclosure from 1 July to 30 September 2021.xlsx
2015 Members' expense disclosure from 1 April to 30 June 2015.xlsx
2018 Members' expense disclosure from 1 April to 30 June 2018.xlsx


In [159]:
final_df.to_excel('test.xlsx')